# **1. Import Library**

In [ ]:
!pip install nltk nlp-id gdown textblob opendatasets

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 69.8 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 57.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 402.6/402.6 kB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 92.3 MB/s eta 0:00:00:00:0100:01
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9656 sha256=88c08073663b4816ed7a2ed9b184d99e2a00f79d8e18b2efa2f06c833d43b272
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.4
    Uninstalling nltk-3.2.4:
      Successfully uninstalled nltk-3.2.4
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2
  Attempting uninstall: huggin

In [ ]:
!gdown --id 1cXaOtuw6xqWL6KDjZqvkzGYsiyyr0X1y

/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1cXaOtuw6xqWL6KDjZqvkzGYsiyyr0X1y
To: /kaggle/working/slang.txt
100%|██████████████████████████████████████| 32.8k/32.8k [00:00<00:00, 53.5MB/s]


In [ ]:
import numpy as np
import pandas as pd
import re
import string
import seaborn as sns
import matplotlib.pyplot as plt
import logging

In [ ]:
import nltk
from textblob import TextBlob
from nltk.corpus import stopwords
from nlp_id.lemmatizer import Lemmatizer
from nlp_id.stopword import StopWord
from nltk import word_tokenize
from nlp_id.postag import PosTag
from wordcloud import WordCloud

nltk.download('punkt_tab')
nltk.download('punkt')
nltk.download('stopwords')
stop_words = set(stopwords.words('indonesian'))
stopword = StopWord()
lemmatizer = Lemmatizer()

[nltk_data] Downloading package punkt_tab to /usr/share/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# **2. Data Retrieval**

In [ ]:
banking = pd.concat([pd.read_csv('/kaggle/input/bnpl-dataset-2024/en - banking.csv'), pd.read_csv('/kaggle/input/bnpl-dataset-2024/id - banking.csv')], ignore_index=True)
ewallet = pd.concat([pd.read_csv('/kaggle/input/bnpl-dataset-2024/en - ewallet.csv'), pd.read_csv('/kaggle/input/bnpl-dataset-2024/id - ewallet.csv')], ignore_index=True)
ecomm = pd.concat([pd.read_csv('/kaggle/input/bnpl-dataset-2024/en - ecomm.csv'), pd.read_csv('/kaggle/input/bnpl-dataset-2024/id - ecomm.csv')], ignore_index=True)
lend = pd.concat([pd.read_csv('/kaggle/input/bnpl-dataset-2024/en - loan.csv'), pd.read_csv('/kaggle/input/bnpl-dataset-2024/id - loan.csv')], ignore_index=True)

In [ ]:
banking['platform_type'] = 'banking'
ewallet['platform_type'] = 'ewallet'
ecomm['platform_type'] = 'ecomm'
lend['platform_type'] = 'lend'

In [ ]:
text = pd.concat([banking, ewallet, ecomm, lend], ignore_index=True)

In [ ]:
text = text.drop(columns=['reviewId','userName','userImage','thumbsUpCount','reviewCreatedVersion','replyContent','repliedAt','appVersion'])

In [ ]:
text.head()

,content,score,at,platform_type
0,banyak spam telepon untuk promosi ajakan buka ...,1,2024-12-11 4:15:16,banking
1,"Aplikasi buruk, ketika saya ingin membayar Pay...",1,2024-10-09 3:25:54,banking
2,"Bagi saya, aplikasi ini bekerja dengan sangat ...",5,2024-03-31 18:46:17,banking
3,Saya menerima SMS dari Bank Allo yang menyatak...,1,2022-12-23 9:14:44,banking
4,Indosat im3 terus mengirimi saya pesan tentang...,1,2023-06-15 10:05:25,banking


In [ ]:
text.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4784 entries, 0 to 4783
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   content        4784 non-null   object
 1   score          4784 non-null   int64 
 2   at             4784 non-null   object
 3   platform_type  4784 non-null   object
dtypes: int64(1), object(3)
memory usage: 149.6+ KB


In [ ]:
text.isnull().sum()
text.duplicated().sum()

0

In [ ]:
text.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4784 entries, 0 to 4783
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   content        4784 non-null   object
 1   score          4784 non-null   int64 
 2   at             4784 non-null   object
 3   platform_type  4784 non-null   object
dtypes: int64(1), object(3)
memory usage: 149.6+ KB


# **3. Data Cleaning**

In [ ]:
def lowercase(review_text):
  low = review_text.lower()
  return low
text['content'] = text['content'].apply(lambda low:lowercase(str(low)))

In [ ]:
def remove_hashtag(review_text, default_replace=""):
  hashtag = re.sub(r'#\w+', default_replace, review_text)
  return hashtag
text['content'] = text['content'].apply(lambda hashtag: remove_hashtag(hashtag))

In [ ]:
def remove_emoji(review_text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"
                           u"\U0001F300-\U0001F5FF"
                           u"\U0001F680-\U0001F6FF"
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           u"\U0001f926-\U0001f937"
                           u"\U00010000-\U0010ffff"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', review_text)
text['content'] = text['content'].apply(lambda emoji: remove_emoji(emoji))

In [ ]:
def remove_superscript(review_text):
  number = re.compile("["u"\U00002070"
                      u"\U000000B9"
                      u"\U000000B2-\U000000B3"
                      u"\U00002074-\U00002079"
                      u"\U0000207A-\U0000207E"
                      u"U0000200D"
                      "]+", flags=re.UNICODE)
  return number.sub(r'', review_text)
text['content'] = text['content'].apply(lambda num: remove_superscript(num))

In [ ]:
def remove_punctuation(review_text, default_text=" "):
  list_punct = string.punctuation
  delete_punct = str.maketrans(list_punct,' '*len(list_punct))
  new_review = ' '.join(review_text.translate(delete_punct).split())
  return new_review
text['content'] = text['content'].apply(lambda punct: remove_punctuation(punct))

In [ ]:
def remove_non_clear_symbol(word):
    pattern = r'[^\w\s]'
    cleaned_word = re.sub(pattern, '', word)
    return cleaned_word
text['content'] = text['content'].apply(lambda word: remove_non_clear_symbol(word))

In [ ]:
def word_repetition(review_text):
  review = re.sub(r'(.)\1+', r'\1\1', review_text)
  return review
text['content'] = text['content'].apply(lambda word: word_repetition(word))

def repetition(review_text):
  repeat = re.sub(r'\b(\w+)(?:\W\1\b)+', r'\1',review_text, flags=re.IGNORECASE)
  return repeat
text['content'] = text['content'].apply(lambda word: repetition(word))

In [ ]:
bannedword = ['wkwk', 'wkwkw','wkwkwk','hihi','hihihii','hihihi','hehehe','hehehehe','hehe',
              'huhu','huhuu','woy', 'tolol', 'anjing', 'babii','babi','terimakasih','anjir',
              'the','najis','kocak','jancuk','amin','aduh','dongo','hehe','aamiin'
              'ikhlas','kocak','lawak','hehe','lah','nya','woi','haha','hahaha','indodana','atome','kredivo',
              'kreivo','kredivoo','akulaku','aplikasi','alhamdulillah', 'shopee', 'allobank', 'allo', 'ovo','gopay','paylater','pay','later','mybca','home','credit']
re_banned_words = re.compile(r"\b(" + "|".join(bannedword) + ")\W", re.I)

def remove_banned_word(toPrint):
    global re_banned_words
    return re_banned_words.sub("", toPrint)
text['content'] = text['content'].apply(lambda banned:remove_banned_word(banned))

In [ ]:
text.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4784 entries, 0 to 4783
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   content        4784 non-null   object
 1   score          4784 non-null   int64 
 2   at             4784 non-null   object
 3   platform_type  4784 non-null   object
dtypes: int64(1), object(3)
memory usage: 149.6+ KB


In [ ]:
text.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4784 entries, 0 to 4783
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   content        4784 non-null   object
 1   score          4784 non-null   int64 
 2   at             4784 non-null   object
 3   platform_type  4784 non-null   object
dtypes: int64(1), object(3)
memory usage: 149.6+ KB


In [ ]:
text.isnull().sum()

content          0
score            0
at               0
platform_type    0
dtype: int64

In [ ]:
def remove_extra_whitespaces(review_text):
  review = re.sub(r'\s+',' ', review_text)
  return review
text['content'] = text['content'].apply(lambda extra_spaces: remove_extra_whitespaces(extra_spaces))

# **4. Data Preprocessing**

In [ ]:
slangs = open('slang.txt',"r",encoding="utf-8", errors='replace')
clear_slangs= []
for newlines in slangs:
  strip_re = newlines.strip("\n")
  split = re.split(r'[:]',strip_re)
  clear_slangs.append(split)
slangs = [[k.strip(), v.strip()] for k,v in clear_slangs]
dict_slangs = {key:values for key,values in slangs}

clean_text = []
for review in text['content']:
  wordlist = TextBlob(review).words
  for k,v in enumerate(wordlist):
    if v in dict_slangs.keys():
      wordlist[k] = dict_slangs[v]
  clean_text.append(' '.join(wordlist))

text['content'] = clean_text

In [ ]:
def word_token(review_text):
  return word_tokenize(review_text)
text['token'] = text['content'].apply(lambda tokenize:word_token(str(tokenize)))

In [ ]:
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('popular')
indonesian_stop = stopwords.words('indonesian')
stopwords_to_remove = ['tidak']
modified_stopwords = [word for word in indonesian_stop if word not in stopwords_to_remove]

stopwords_indo = pd.DataFrame(indonesian_stop, columns=['stopwords_indonenesia'])
stopwords_indo.to_excel('stopwords_indonesian.xlsx', index=False)

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     /usr/share/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     /usr/share/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     /usr/share/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     /usr/share/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to
[nltk_data]    |     /usr/share/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_r

In [ ]:
def remove_stopwords(review_text, modified_stopwords):
  tokenize = []
  for token in review_text:
    if token not in modified_stopwords:
      tokenize.append(token)
  return tokenize

text['token'] = text['token'].apply(lambda stop: remove_stopwords(stop, indonesian_stop))

In [ ]:
def lemmatize_text(review_text):
    stop_token = ' '.join(review_text)
    result_lemma = lemmatizer.lemmatize(stop_token)
    return result_lemma

text['lemmatized'] =text['token'].apply(lambda stem: lemmatize_text(stem))

In [ ]:
text.info()
text.tail()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4784 entries, 0 to 4783
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   content        4784 non-null   object
 1   score          4784 non-null   int64 
 2   at             4784 non-null   object
 3   platform_type  4784 non-null   object
 4   token          4784 non-null   object
 5   lemmatized     4784 non-null   object
dtypes: int64(1), object(5)
memory usage: 224.4+ KB


,content,score,at,platform_type,token,lemmatized
4779,lumayan sangat membantu,5,2023-08-03 10:59:59,lend,"[lumayan, membantu]",lumayan bantu
4780,jos,5,2022-07-21 09:04:50,lend,[jos],jos
4781,pengajuan credit,5,2021-10-28 15:55:01,lend,"[pengajuan, credit]",aju credit
4782,paylater,5,2021-05-20 22:28:05,lend,[paylater],paylater
4783,tidak bisa,5,2024-06-15 14:59:02,lend,[],


In [ ]:
def join_text(review_text):
    stop_token = ' '.join(review_text)
    return stop_token
text['clean'] = text['token'] .apply(lambda tokens: join_text(tokens))

In [ ]:
def count_words(sentence):
    return len(sentence.split())
text = text[text['clean'].apply(count_words) >= 3]

In [ ]:
text['at'] = pd.to_datetime(text['at'])
text['period'] = text['at'].dt.to_period('M')

In [ ]:
text.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4488 entries, 0 to 4778
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   content        4488 non-null   object        
 1   score          4488 non-null   int64         
 2   at             4488 non-null   datetime64[ns]
 3   platform_type  4488 non-null   object        
 4   token          4488 non-null   object        
 5   lemmatized     4488 non-null   object        
 6   clean          4488 non-null   object        
 7   period         4488 non-null   period[M]     
dtypes: datetime64[ns](1), int64(1), object(5), period[M](1)
memory usage: 315.6+ KB
